In [72]:
import pandas as pd

from risk_icu.preprocessing import preprocessing_24_hour

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score

from imblearn.over_sampling import SMOTE

## Data

In [73]:
data = pd.read_csv("raw_data/training_v2.csv")
X, y = preprocessing_24_hour(data)

/Users/franciscochaves/code/DomNau1001/Risk_ICU/risk_icu/preprocessing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_post_cats[ohe.get_feature_names_out()] = ohe.transform(X_post_cats[["ethnicity"]])
/Users/franciscochaves/code/DomNau1001/Risk_ICU/risk_icu/preprocessing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_post_cats[ohe.get_feature_names_out()] = ohe.transform(X_post_cats[["ethnicity"]])
/Users/franciscochaves/code/DomNau1001/Risk_ICU/risk_icu/preprocessing.py:70: SettingWithCopyW

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=42)

In [75]:
y_train.value_counts()

0
0    46432
1     5047
Name: count, dtype: int64

In [76]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [77]:
y_train.value_counts()

0
0    37145
1     4038
Name: count, dtype: int64

## Data Balancing

In [78]:
X_res, y_res = SMOTE(random_state=42).fit_resample(X_train, y_train)

In [79]:
y_res.value_counts()

0
0    37145
1    37145
Name: count, dtype: int64

## SVC

In [51]:
svc = SVC(kernel = "sigmoid", C = 9.18, coef0=0.14, gamma = "scale", probability = True, class_weight="balanced")
svc.fit(X_res, y_res)

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=9.18, class_weight='balanced', coef0=0.14, kernel='sigmoid',
    probability=True)

In [7]:
ada_svc = AdaBoostClassifier(svc, n_estimators=50)

ada_svc.fit(X_res, y_res)

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [7]:
predictions = svc.predict(X_test)
recall_svc = recall_score(y_test, predictions, average='binary')
recall_svc

0.5977808599167822

In [9]:
y_score = svc.predict_proba(X_test)[:, 1]
y_true = y_test
auc_score = roc_auc_score(y_true, y_score)
print("ROC AUC Score:", auc_score)

ROC AUC Score: 0.6470425172557193


## Naive Bayes

In [80]:
nb = GaussianNB(var_smoothing=5.964585534596374e-06)
nb.fit(X_res, y_res)

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(var_smoothing=5.964585534596374e-06)

In [81]:
ada_nb = AdaBoostClassifier(nb, n_estimators=50)
ada_nb.fit(X_res, y_res)

/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=GaussianNB(var_smoothing=5.964585534596374e-06))

In [82]:
predictions = ada_nb.predict(X_test)
recall_nb = recall_score(y_test, predictions)
recall_nb

0.8344891354600092

In [83]:
y_score = ada_nb.predict_proba(X_test)[:, 1]
y_true = y_test
auc_score = roc_auc_score(y_true, y_score)
print("ROC AUC Score:", auc_score)

ROC AUC Score: 0.504176383535802


## Decision Tree + Adaptive Boosting

In [84]:
tree = DecisionTreeClassifier(class_weight = "balanced", min_samples_split=12, min_samples_leaf=6, max_leaf_nodes=45, max_features=None, max_depth = 2, criterion = "entropy", splitter = "random", min_impurity_decrease=0.0)
tree.fit(X_res, y_res)

ada_tree = AdaBoostClassifier(tree, n_estimators=100, learning_rate = 0.2, algorithm = "SAMME.R",random_state=42)
ada_tree.fit(X_res, y_res)


/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/franciscochaves/.pyenv/versions/3.10.6/envs/Risk_ICU/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                    criterion='entropy',
                                                    max_depth=2,
                                                    max_leaf_nodes=45,
                                                    min_samples_leaf=6,
                                                    min_samples_split=12,
                                                    splitter='random'),
                   learning_rate=0.2, n_estimators=100, random_state=42)

In [85]:
predictions = ada_tree.predict(X_test)
recall_dt = recall_score(y_test, predictions, average='binary')
recall_dt


0.6999537679149329

In [86]:
y_score = ada_tree.predict_proba(X_test)[:, 1]
y_true = y_test
auc_score = roc_auc_score(y_true, y_score)
print("ROC AUC Score:", auc_score)

ROC AUC Score: 0.8244564709818162


## Ensemble Model

#1. -> Voting Classifier -> ada_svc, ada_nb, ada_tree
#2. -> Stacking Classifier -> ada_svc, ada_nb, ada_tree

## Voting Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

classifier1 = ada_svc
classifier2 = ada_nb
classifier3 = ada_tree

In [ ]:
voting_classifier = VotingClassifier(
    estimators=[('svc', classifier1), ('nb', classifier2), ('tree', classifier3)],
    voting='hard'
)

voting_classifier.fit(X_res, y_res)
predictions = voting_classifier.predict(X_test)

recall_hard = recall_score(y_test, predictions, average='binary')
recall_hard

In [ ]:
voting_classifier = VotingClassifier(
    estimators=[('svc', classifier1), ('nb', classifier2), ('tree', classifier3)],
    voting='soft'
)

voting_classifier.fit(X_res, y_res)
predictions = voting_classifier.predict(X_test)

recall_soft = recall_score(y_test, predictions, average='binary')
recall_soft

## Stacking